In [ ]:
import pandas as pd
import numpy as np
import nltk

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     /home/devansh/nltk_data...
[nltk_data]    |   Unzipping corpora/cmudict.zip.
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     /home/devansh/nltk_data...
[nltk_data]    |   Unzipping corpora/gazetteers.zip.
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     /home/devansh/nltk_data...
[nltk_data]    |   Unzipping corpora/genesis.zip.
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     /home/devansh/nltk_data...
[nltk_data]    |   Unzipping corpora/gutenberg.zip.
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     /home/devansh/nltk_data...
[nltk_data]    |   Unzipping corpora/inaugural.zip.
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /home/devansh/nltk_data...
[nltk_data]    |   Unzipping corpora/movie_reviews.zip.
[nltk_data]    | Downloading packag

True

In [20]:
nltk.download('popular')

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     /home/devansh/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     /home/devansh/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     /home/devansh/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     /home/devansh/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     /home/devansh/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /home/devansh/nltk_data...
[nltk_data]    |   Package movie_reviews is already

True

In [4]:
train = pd.read_csv('data/train.csv')
train

,id,query,product_title,product_description,median_relevance,relevance_variance
0,1,bridal shower decorations,Accent Pillow with Heart Design - Red/Black,Red satin accent pillow embroidered with a hea...,1,0.000
1,2,led christmas lights,Set of 10 Battery Operated Multi LED Train Chr...,Set of 10 Battery Operated Train Christmas Lig...,4,0.000
2,4,projector,ViewSonic Pro8200 DLP Multimedia Projector,NaN,4,0.471
3,5,wine rack,Concept Housewares WR-44526 Solid-Wood Ceiling...,"Like a silent and sturdy tree, the Southern En...",4,0.000
4,7,light bulb,Wintergreen Lighting Christmas LED Light Bulb ...,"WTGR1011\nFeatures\nNickel base, 60,000 averag...",2,0.471
...,...,...,...,...,...,...
10153,32655,plantronics corded headset,Plantronics TriStar H81N - headset,With four sizes of user-selectable earbuds and...,4,0.471
10154,32659,spiderman,Marvel Legends Infinite Series Amazing Spider-...,NaN,2,0.471
10155,32663,playstation vita system,MLB 15: The Show (PlayStation Vita),An essential part of every fan’s season and Th...,2,1.247
10156,32666,flower bulbs,Junkyard Findings Vintage Trinkets-Small Typo ...,PRIMA FLOWERS-Junkyard Findings Vintage Trinke...,1,0.471


## Data Cleaning and Feature engineering
Let's start by cleaning the description of HTML tags and newlines

In [14]:
import re

pattern = r"<\/?\w*>|[\s]&[^ ]|\n"
def clean_html(inp):
    return re.sub(pattern, '', inp)

train['product_description'] = train['product_description'].map(clean_html, na_action='ignore')

In [17]:
# fix typos
from symspellpy.symspellpy import SymSpell, Verbosity
import pkg_resources

# Initialize SymSpell object
sym_spell = SymSpell(max_dictionary_edit_distance=2, prefix_length=7)

# Load the frequency dictionary
dictionary_path = pkg_resources.resource_filename(
    "symspellpy", "frequency_dictionary_en_82_765.txt"
)
sym_spell.load_dictionary(dictionary_path, term_index=0, count_index=1)

def correct_text(text):
    # symspellpy expects a string, so convert NaN or non-string to empty string
    if not isinstance(text, str):
        return text
    suggestions = sym_spell.lookup(text, Verbosity.CLOSEST, max_edit_distance=2)
    if suggestions:
        return suggestions[0].term
    else:
        return text
    
train['query'] = train['query'].apply(correct_text)

In [ ]:
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
stemmer = SnowballStemmer(language='english')

# now I'll apply stemming and remove stop words
# snowball stemmer removes stopwords by default
def stem_and_remove_stopwords(text):
    tokenized = word_tokenize(text)
    processed = [[stemmer.stem(word) for word in tokenized if not word in stop_words]]
    return ' '.join(processed)